In [10]:
from torch.utils.data import Dataset
import os

In [9]:
from torch.utils.data import Dataset
class predictorData(Dataset):
    
    def __init__(self, root_dir):
    
    def __getitem__(self, idx):

IndentationError: expected an indented block after function definition on line 3 (3945838994.py, line 5)